In [ ]:
# Install required packages
!pip install librosa
!pip install pydub
!apt-get install ffmpeg  # FFmpeg is required to handle mp3 files in pydub
!pip install yt-dlp
# Install required libraries with modern versions
!pip install torch torchvision torchaudio
!pip install torchlibrosa
!pip install h5py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB

In [ ]:
!pip install pydub tqdm
!sudo apt install ffmpeg rubberband-cli

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following NEW packages will be installed:
  rubberband-cli
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 87.5 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 rubberband-cli amd64 2.0.0-2 [87.5 kB]
Fetched 87.5 kB in 1s (121 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselect

In [ ]:
import yt_dlp
from pydub import AudioSegment
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import torch
import sys
import json
import os
from IPython.display import Audio
import random
import soundfile as sf
from pathlib import Path
import subprocess
from tqdm import tqdm
import shutil
from torch.utils.data import Dataset, Subset
import gc
from sklearn.model_selection import train_test_split
import torchaudio
import h5py
import time
import wave

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def download_audio_from_playlist(playlist_url, n, start_video=1, download_path='/content/', folder=''):
    # Construct the full output path
    output_path = os.path.join(download_path, folder)
    os.makedirs(output_path, exist_ok=True)  # Create the directory if it doesn't exist

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'playliststart': start_video,
        'playlistend': start_video + n - 1,
        'quiet': False,
        'extract_audio': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            print(f"Downloading {n} audios starting from video {start_video} in playlist {playlist_url} into {output_path}...")
            ydl.download([playlist_url])
            print("Download complete!")
    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
playlist_url = "https://youtube.com/playlist?list=PLXKO_mcZGJ8ylGHogxGUROkVev_PVAZ5N&feature=shared"
download_path = '/content/drive/My Drive/CSE493G1/Final Project/Datasets'
download_audio_from_playlist(playlist_url, n=120, download_path=download_path, folder="Country")

[youtube:tab] Extracting URL: https://youtube.com/playlist?list=PLXKO_mcZGJ8ylGHogxGUROkVev_PVAZ5N&feature=shared
[youtube:tab] PLXKO_mcZGJ8ylGHogxGUROkVev_PVAZ5N: Downloading webpage
[youtube:tab] PLXKO_mcZGJ8ylGHogxGUROkVev_PVAZ5N: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Country
[youtube:tab] PLXKO_mcZGJ8ylGHogxGUROkVev_PVAZ5N page 1: Downloading API JSON
[youtube:tab] Playlist Country: Downloading 120 items of 120
[download] Downloading item 1 of 120
[youtube] Extracting URL: https://www.youtube.com/watch?v=U3trDjH6djw
[youtube] U3trDjH6djw: Downloading webpage
[youtube] U3trDjH6djw: Downloading tv client config
[youtube] U3trDjH6djw: Downloading tv player API JSON
[youtube] U3trDjH6djw: Downloading ios player API JSON
[youtube] U3trDjH6djw: Downloading m3u8 information
[info] U3trDjH6djw: Downloading 1 format(s): 251
[download] Destination: /content/drive/My Drive/CSE493G1/Final Project/Datasets/Country/Sierra Ferrell -  The Garden (

In [ ]:
def convert_and_splice_mp3s(
    folder, x=90, y=10, z=20, splice_factor=8,
    splice_duration=20,
    output_base="/content/drive/MyDrive/CSE493G1/Final Project/Datasets"
):
    folder = Path(folder)
    mp3_files = list(folder.glob("*.mp3"))
    random.shuffle(mp3_files)

    partitions = {
        'TrainWav': mp3_files[:x],
        'ValWav': mp3_files[x:x + y],
        'TestWav': mp3_files[x + y:x + y + z]
    }

    for subset, files in partitions.items():
        output_dir = Path(output_base) / subset
        output_dir.mkdir(parents=True, exist_ok=True)

        for file in files:
            try:
                # Convert MP3 to WAV
                audio = AudioSegment.from_mp3(file)
                audio = audio.set_frame_rate(32000)
                wav_path = file.with_suffix('.wav')
                audio.export(wav_path, format="wav")

                # Load with librosa
                y, sr = librosa.load(wav_path, sr=None)
                duration_sec = librosa.get_duration(y=y, sr=sr)

                # Compute splicing
                num_splices = int(duration_sec // splice_factor)
                splice_len = int(splice_duration * sr)
                total_len = len(y)
                stride = max(1, (total_len - splice_len) // max(num_splices - 1, 1))

                for i in range(num_splices):
                    start = i * stride
                    end = start + splice_len
                    if end > total_len:
                        break
                    splice = y[start:end]

                    # Write splice directly to final output file
                    base_name = f"{file.stem}_splice{i}.wav"
                    out_path = output_dir / base_name
                    sf.write(out_path, splice, sr)

                # Clean up intermediate WAV
                if wav_path.exists():
                    wav_path.unlink()

                # Optionally delete the intermediate WAV
                if wav_path.exists():
                    os.remove(wav_path)

            except Exception as e:
                print(f"Failed to process {file}: {e}")

In [ ]:
convert_and_splice_mp3s('/content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass', x=90, y=10, z=20)

In [ ]:
def delete_wav_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            file_path = os.path.join(folder_path, filename)
            try:
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Failed to delete {file_path}: {e}")

delete_wav_files('/content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass')

Deleted: /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/Bronwyn Keith-Hynes ＂Will You Ever Be Mine＂ (Official Music Video).wav
Deleted: /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/We Can't Be Darlings Anymore.wav
Deleted: /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/Blue Ridge Cabin Home.wav
Deleted: /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/Lovin' Pretty Women.wav
Deleted: /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/In Dreams I Go Back.wav
Deleted: /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/I Am A Man Of Constant Sorrow (With Band).wav
Deleted: /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/Black Mountain Rag.wav
Deleted: /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/Doc Watson - Streamlined Cannonball (Official Visualier).wav
Deleted: /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/Lester Flatt and Earl Scru

In [ ]:
class WesternDataset(Dataset):
    def __init__(self, data):
        # data is a list of (waveform, label) tuples
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        waveform, label = self.data[idx]
        return waveform, label


In [ ]:
def num_train_files_in_genre(dir, genre):
  folder = os.path.join(dir, genre, "TrainWav")
  return len([f for f in os.listdir(folder) if f.endswith(".wav")])

num_bluegrass_train = num_train_files_in_genre(dir, "Bluegrass")
num_country_train = num_train_files_in_genre(dir, "Country")
num_folk_train = num_train_files_in_genre(dir, "Folk")

print(num_bluegrass_train)
print(num_country_train)
print(num_folk_train)

2222
2269
2264


In [ ]:
def save_train_dataset_to_h5(dataset, save_path, split_num):
    with h5py.File(f"{save_path}/train_{split_num}", "w") as h5f:
        labels = []
        dt = h5py.vlen_dtype(np.dtype('float32'))
        wave_group = h5f.create_dataset("waveforms", (len(dataset),), dtype=dt)

        for i, (waveform, label) in enumerate(dataset):
            waveform_np = waveform.numpy().flatten().astype(np.float32)
            wave_group[i] = waveform_np
            labels.append(label)

        h5f.create_dataset("labels", data=np.array(labels, dtype=np.int64))

    print(f"Saved {len(dataset)} items to {save_path}")


In [ ]:
def save_val_dataset_to_h5(dataset, save_path):
    with h5py.File(f"{save_path}/val", "w") as h5f:
        labels = []
        dt = h5py.vlen_dtype(np.dtype('float32'))
        wave_group = h5f.create_dataset("waveforms", (len(dataset),), dtype=dt)

        for i, (waveform, label) in enumerate(dataset):
            waveform_np = waveform.numpy().flatten().astype(np.float32)
            wave_group[i] = waveform_np
            labels.append(label)

        h5f.create_dataset("labels", data=np.array(labels, dtype=np.int64))

    print(f"Saved {len(dataset)} items to {save_path}")

In [ ]:
def save_test_dataset_to_h5(dataset, save_path):
    with h5py.File(f"{save_path}/test", "w") as h5f:
        labels = []
        dt = h5py.vlen_dtype(np.dtype('float32'))
        wave_group = h5f.create_dataset("waveforms", (len(dataset),), dtype=dt)

        for i, (waveform, label) in enumerate(dataset):
            waveform_np = waveform.numpy().flatten().astype(np.float32)
            wave_group[i] = waveform_np
            labels.append(label)

        h5f.create_dataset("labels", data=np.array(labels, dtype=np.int64))

    print(f"Saved {len(dataset)} items to {save_path}")

In [ ]:
# Dataset root directory in your mounted Google Drive
dir = "/content/drive/MyDrive/CSE493G1/Final Project/Datasets"
num_splits = 5

# Mapping of folder to label
genre_map = {
    "Bluegrass": 0,
    "Country": 1,
    "Folk": 2
}

for split in range(num_splits):
    samples = []
    for genre, label in genre_map.items():
        genre_train_dir = os.path.join(dir, genre, "TrainWav")
        for idx, filename in enumerate(os.listdir(genre_train_dir)):
            if idx > 20:
                break
            if idx % num_splits != split:
                continue
            if not filename.endswith(".wav"):
                continue
            wav_filepath = os.path.join(genre_train_dir, filename)
            print(f"processing {wav_filepath}")
            try:
                with wave.open(wav_filepath, 'rb') as wf:
                    print(f"{wav_filepath} duration: {wf.getnframes()/wf.getframerate()}s")
            except Exception as e:
                print(f"Corrupted or invalid WAV file: {wav_filepath} ({e})")
            try:
                waveform, sr = torchaudio.load(wav_filepath)
                samples.append((waveform, label))
            except Exception as e:
                print("Load failed with error:", e)

    split_dataset = WesternDataset(samples)
    save_train_dataset_to_h5(split_dataset, dir, split)



processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/TrainWav/Movin' on Down That Line_splice2.wav
/content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/TrainWav/Movin' on Down That Line_splice2.wav duration: 20.0s
processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/TrainWav/Movin' on Down That Line_splice7.wav
/content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/TrainWav/Movin' on Down That Line_splice7.wav duration: 20.0s
processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/TrainWav/Movin' on Down That Line_splice12.wav
/content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/TrainWav/Movin' on Down That Line_splice12.wav duration: 20.0s
processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/TrainWav/Movin' on Down That Line_splice17.wav
/content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/TrainWav/Movin' on Down That Line_splice17.wav duration: 20.0s


In [ ]:
samples = []
for genre, label in genre_map.items():
    genre_train_dir = os.path.join(dir, genre, "ValWav")
    for idx, filename in enumerate(os.listdir(genre_train_dir)):
        if not filename.endswith(".wav"):
            continue
        wav_filepath = os.path.join(genre_train_dir, filename)
        print(f"processing {wav_filepath}")
        try:
            time.sleep(0.1)
            waveform, sr = torchaudio.load(wav_filepath)
        except Exception as e:
            print("Load failed with error:", e)
        samples.append((waveform, label))
val_dataset = WesternDataset(samples)
save_val_dataset_to_h5(val_dataset, dir)

processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/ValWav/Norman Blake & Tony Rice - New River Train (Official Audio)_splice0.wav
processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/ValWav/Norman Blake & Tony Rice - New River Train (Official Audio)_splice1.wav
processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/ValWav/Norman Blake & Tony Rice - New River Train (Official Audio)_splice2.wav
processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/ValWav/Norman Blake & Tony Rice - New River Train (Official Audio)_splice3.wav
processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/ValWav/Norman Blake & Tony Rice - New River Train (Official Audio)_splice4.wav
processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Bluegrass/ValWav/Norman Blake & Tony Rice - New River Train (Official Audio)_splice5.wav
processing /content/drive/MyDrive/CSE493G1/Final Project/Datasets/Blue

In [ ]:
samples = []
for genre, label in genre_map.items():
    genre_train_dir = os.path.join(dir, genre, "TestWav")
    for idx, filename in enumerate(os.listdir(genre_train_dir)):
        if not filename.endswith(".wav"):
            continue
        wav_filepath = os.path.join(genre_train_dir, filename)
        print(f"processing {wav_filepath}")
        try:
            time.sleep(0.1)
            waveform, sr = torchaudio.load(wav_filepath)
        except Exception as e:
            print("Load failed with error:", e)
        samples.append((waveform, label))
test_dataset = WesternDataset(samples)
save_test_dataset_to_h5(test_dataset, dir)